**Set up project config**

In [2]:
## You need to change the project ID to match your GCP
PROJECT_ID = "halogen-rarity-312520"

In [3]:
import os
from google.colab import auth

#sets dateset
DATASET_PROJECT_ID = 'amsterdamumcdb-data'
DATASET_ID = 'ams102'
LOCATION = 'eu'

#all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')

Authenticated


**Set up Google Cloud Platform / Big Query config**

In [4]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

#sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config

#sets client options job configuration
client_options = {}
client_options['location'] = LOCATION
magics.context.bigquery_client_options = client_options

In [5]:
import pandas as pd
import numpy as np
import re

config_gbq = {'query': 
          {'defaultDataset': {
              "datasetId": DATASET_ID, 
              "projectId": DATASET_PROJECT_ID
              },
           'Location': LOCATION}
           }

In [6]:
pd.set_option('display.max_rows', 100)

In [7]:
diagnoses_sql = '''
WITH diagnosis_groups AS (
    SELECT admissionid,
        item, 
        value as diagnosis_group,
        valueid as diagnosis_group_id,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY 
            CASE --prefer NICE > APACHE IV > II > D
                WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen                
                WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis        
                WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
            END DESC,
        measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        --MAIN GROUP - LEVEL 0
        13110, --D_Hoofdgroep
        16651, --DMC_Hoofdgroep, Medium Care
        
        18588, --Apache II Hoofdgroep
        16997, --APACHE IV Groepen
        
        18669, --NICE APACHEII diagnosen
        18671 --NICE APACHEIV diagnosen
    )
),diagnosis_subgroups AS (
    SELECT admissionid,
        item, 
        value as diagnosis_subgroup,
        valueid as diagnosis_subgroup_id,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        --SUB GROUP - LEVEL 1
        13111, --D_Subgroep_Thoraxchirurgie
        16669, --DMC_Subgroep_Thoraxchirurgie
        13112, --D_Subgroep_Algemene chirurgie
        16665, --DMC_Subgroep_Algemene chirurgie
        13113, --D_Subgroep_Neurochirurgie
        16667, --DMC_Subgroep_Neurochirurgie
        13114, --D_Subgroep_Neurologie
        16668, --DMC_Subgroep_Neurologie
        13115, --D_Subgroep_Interne geneeskunde
        16666 --DMC_Subgroep_Interne geneeskunde
    )
), diagnoses AS (
    SELECT admissionid,
        item, 
        value as diagnosis,
        CASE
            WHEN itemid IN (
                --SURGICAL
                13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
                16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
                13117, --D_Thoraxchirurgie_Cardio anders
                16672, --DMC_Thoraxchirurgie_Cardio anders
                13118, --D_Thoraxchirurgie_Aorta chirurgie
                16670, --DMC_Thoraxchirurgie_Aorta chirurgie
                13119, --D_Thoraxchirurgie_Pulmonale chirurgie
                16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie

                --Not surgical: 13141, --D_Algemene chirurgie_Algemeen   
                --Not surgical: 16642, --DMC_Algemene chirurgie_Algemeen
                13121, --D_Algemene chirurgie_Buikchirurgie
                16643, --DMC_Algemene chirurgie_Buikchirurgie
                13123, --D_Algemene chirurgie_Endocrinologische chirurgie
                16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
                13145, --D_Algemene chirurgie_KNO/Overige
                16645, --DMC_Algemene chirurgie_KNO/Overige
                13125, --D_Algemene chirurgie_Orthopedische chirurgie
                16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
                13122, --D_Algemene chirurgie_Transplantatie chirurgie
                16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
                13124, --D_Algemene chirurgie_Trauma
                16648, --DMC_Algemene chirurgie_Trauma
                13126, --D_Algemene chirurgie_Urogenitaal
                16649, --DMC_Algemene chirurgie_Urogenitaal
                13120, --D_Algemene chirurgie_Vaatchirurgie
                16650, --DMC_Algemene chirurgie_Vaatchirurgie

                13128, --D_Neurochirurgie _Vasculair chirurgisch
                16661, --DMC_Neurochirurgie _Vasculair chirurgisch
                13129, --D_Neurochirurgie _Tumor chirurgie
                16660, --DMC_Neurochirurgie _Tumor chirurgie
                13130, --D_Neurochirurgie_Overige
                16662, --DMC_Neurochirurgie_Overige

                18596, --Apache II Operatief  Gastr-intenstinaal
                18597, --Apache II Operatief Cardiovasculair
                18598, --Apache II Operatief Hematologisch
                18599, --Apache II Operatief Metabolisme
                18600, --Apache II Operatief Neurologisch
                18601, --Apache II Operatief Renaal
                18602, --Apache II Operatief Respiratoir

                17008, --APACHEIV Post-operative cardiovascular
                17009, --APACHEIV Post-operative gastro-intestinal
                17010, --APACHEIV Post-operative genitourinary
                17011, --APACHEIV Post-operative hematology
                17012, --APACHEIV Post-operative metabolic
                17013, --APACHEIV Post-operative musculoskeletal /skin
                17014, --APACHEIV Post-operative neurologic
                17015, --APACHEIV Post-operative respiratory
                17016, --APACHEIV Post-operative transplant
                17017 --APACHEIV Post-operative trauma

            ) THEN 1
            WHEN itemid = 18669 AND valueid BETWEEN 1 AND 26 THEN 1 --NICE APACHEII diagnosen
            WHEN itemid = 18671 AND valueid BETWEEN 222 AND 452 THEN 1 --NICE APACHEIV diagnosen
            ELSE 0
        END AS surgical,
        valueid as diagnosis_id,
        CASE 
                WHEN itemid = 18671 THEN 'NICE APACHE IV'
                WHEN itemid = 18669 THEN 'NICE APACHE II'
                WHEN itemid BETWEEN 16998 AND 17017 THEN 'APACHE IV'
                WHEN itemid BETWEEN 18589 AND 18602 THEN 'APACHE II'
                WHEN itemid BETWEEN 13116 AND 13145 THEN 'Legacy ICU'
                WHEN itemid BETWEEN 16642 AND 16673 THEN 'Legacy MCU'
        END AS diagnosis_type,
        ROW_NUMBER() OVER(PARTITION BY admissionid
        ORDER BY 
            CASE --prefer NICE > APACHE IV > II > D
                WHEN itemid = 18671 THEN 6 --NICE APACHEIV diagnosen
                WHEN itemid = 18669 THEN 5 --NICE APACHEII diagnosen                
                WHEN itemid BETWEEN 16998 AND 17017 THEN 4 --APACHE IV diagnosis        
                WHEN itemid BETWEEN 18589 AND 18602 THEN 3 --APACHE II diagnosis
                WHEN itemid BETWEEN 13116 AND 13145 THEN 2 --D diagnosis ICU
                WHEN itemid BETWEEN 16642 AND 16673 THEN 1 --DMC diagnosis Medium Care
            END DESC,
            measuredat DESC) AS rownum
    FROM listitems
    WHERE itemid IN (
        -- Diagnosis - LEVEL 2
        --SURGICAL
        13116, --D_Thoraxchirurgie_CABG en Klepchirurgie
        16671, --DMC_Thoraxchirurgie_CABG en Klepchirurgie
        13117, --D_Thoraxchirurgie_Cardio anders
        16672, --DMC_Thoraxchirurgie_Cardio anders
        13118, --D_Thoraxchirurgie_Aorta chirurgie
        16670, --DMC_Thoraxchirurgie_Aorta chirurgie
        13119, --D_Thoraxchirurgie_Pulmonale chirurgie
        16673, --DMC_Thoraxchirurgie_Pulmonale chirurgie
        
        13141, --D_Algemene chirurgie_Algemeen   
        16642, --DMC_Algemene chirurgie_Algemeen
        13121, --D_Algemene chirurgie_Buikchirurgie
        16643, --DMC_Algemene chirurgie_Buikchirurgie
        13123, --D_Algemene chirurgie_Endocrinologische chirurgie
        16644, --DMC_Algemene chirurgie_Endocrinologische chirurgie
        13145, --D_Algemene chirurgie_KNO/Overige
        16645, --DMC_Algemene chirurgie_KNO/Overige
        13125, --D_Algemene chirurgie_Orthopedische chirurgie
        16646, --DMC_Algemene chirurgie_Orthopedische chirurgie
        13122, --D_Algemene chirurgie_Transplantatie chirurgie
        16647, --DMC_Algemene chirurgie_Transplantatie chirurgie
        13124, --D_Algemene chirurgie_Trauma
        16648, --DMC_Algemene chirurgie_Trauma
        13126, --D_Algemene chirurgie_Urogenitaal
        16649, --DMC_Algemene chirurgie_Urogenitaal
        13120, --D_Algemene chirurgie_Vaatchirurgie
        16650, --DMC_Algemene chirurgie_Vaatchirurgie

        13128, --D_Neurochirurgie _Vasculair chirurgisch
        16661, --DMC_Neurochirurgie _Vasculair chirurgisch
        13129, --D_Neurochirurgie _Tumor chirurgie
        16660, --DMC_Neurochirurgie _Tumor chirurgie
        13130, --D_Neurochirurgie_Overige
        16662, --DMC_Neurochirurgie_Overige
        
        18596, --Apache II Operatief  Gastr-intenstinaal
        18597, --Apache II Operatief Cardiovasculair
        18598, --Apache II Operatief Hematologisch
        18599, --Apache II Operatief Metabolisme
        18600, --Apache II Operatief Neurologisch
        18601, --Apache II Operatief Renaal
        18602, --Apache II Operatief Respiratoir
        
        17008, --APACHEIV Post-operative cardiovascular
        17009, --APACHEIV Post-operative gastro-intestinal
        17010, --APACHEIV Post-operative genitourinary
        17011, --APACHEIV Post-operative hematology
        17012, --APACHEIV Post-operative metabolic
        17013, --APACHEIV Post-operative musculoskeletal /skin
        17014, --APACHEIV Post-operative neurologic
        17015, --APACHEIV Post-operative respiratory
        17016, --APACHEIV Post-operative transplant
        17017, --APACHEIV Post-operative trauma

        --MEDICAL
        13133, --D_Interne Geneeskunde_Cardiovasculair
        16653, --DMC_Interne Geneeskunde_Cardiovasculair
        13134, --D_Interne Geneeskunde_Pulmonaal
        16658, --DMC_Interne Geneeskunde_Pulmonaal
        13135, --D_Interne Geneeskunde_Abdominaal
        16652, --DMC_Interne Geneeskunde_Abdominaal
        13136, --D_Interne Geneeskunde_Infectieziekten
        16655, --DMC_Interne Geneeskunde_Infectieziekten
        13137, --D_Interne Geneeskunde_Metabool
        16656, --DMC_Interne Geneeskunde_Metabool
        13138, --D_Interne Geneeskunde_Renaal
        16659, --DMC_Interne Geneeskunde_Renaal
        13139, --D_Interne Geneeskunde_Hematologisch
        16654, --DMC_Interne Geneeskunde_Hematologisch
        13140, --D_Interne Geneeskunde_Overige
        16657, --DMC_Interne Geneeskunde_Overige

        13131, --D_Neurologie_Vasculair neurologisch
        16664, --DMC_Neurologie_Vasculair neurologisch
        13132, --D_Neurologie_Overige
        16663, --DMC_Neurologie_Overige 
        13127, --D_KNO/Overige
        
        18589, --Apache II Non-Operatief Cardiovasculair
        18590, --Apache II Non-Operatief Gastro-intestinaal
        18591, --Apache II Non-Operatief Hematologisch
        18592, --Apache II Non-Operatief Metabolisme
        18593, --Apache II Non-Operatief Neurologisch
        18594, --Apache II Non-Operatief Renaal
        18595, --Apache II Non-Operatief Respiratoir
        
        16998, --APACHE IV Non-operative cardiovascular
        16999, --APACHE IV Non-operative Gastro-intestinal
        17000, --APACHE IV Non-operative genitourinary
        17001, --APACHEIV  Non-operative haematological
        17002, --APACHEIV  Non-operative metabolic
        17003, --APACHEIV Non-operative musculo-skeletal
        17004, --APACHEIV Non-operative neurologic
        17005, --APACHEIV Non-operative respiratory
        17006, --APACHEIV Non-operative transplant
        17007, --APACHEIV Non-operative trauma
        
        --NICE: surgical/medical combined in same parameter
        18669, --NICE APACHEII diagnosen
        18671 --NICE APACHEIV diagnosen
    )
), sepsis AS (
    SELECT
        admissionid,
        CASE valueid
            WHEN 1 THEN 1 --'Ja'
            WHEN 2 THEN 0 --'Nee'
        END as sepsis_at_admission,
        ROW_NUMBER() OVER(
            PARTITION BY 
                admissionid
            ORDER BY 
                measuredat DESC) AS rownum
    FROM listitems
    WHERE 
        itemid = 15808
), sepsis_antibiotics AS ( --non prophylactic antibiotics
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_antibiotics_bool,
        STRING_AGG(DISTINCT item, '; ') AS sepsis_antibiotics_given
    FROM drugitems
    WHERE 
        itemid IN (
            6834, --Amikacine (Amukin)
            6847, --Amoxicilline (Clamoxyl/Flemoxin)
            6871, --Benzylpenicilline (Penicilline)
            6917, --Ceftazidim (Fortum)
            --6919, --Cefotaxim (Claforan) -> prophylaxis
            6948, --Ciprofloxacine (Ciproxin)
            6953, --Rifampicine (Rifadin)
            6958, --Clindamycine (Dalacin)
            7044, --Tobramycine (Obracin)
            --7064, --Vancomycine -> prophylaxis for valve surgery
            7123, --Imipenem (Tienam)
            7185, --Doxycycline (Vibramycine)
            --7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
            --7208, --Erythromycine (Erythrocine) -> often used for gastroparesis
            7227, --Flucloxacilline (Stafoxil/Floxapen)
            7231, --Fluconazol (Diflucan)
            7232, --Ganciclovir (Cymevene)
            7233, --Flucytosine (Ancotil)
            7235, --Gentamicine (Garamycin)
            7243, --Foscarnet trinatrium (Foscavir)
            7450, --Amfotericine B (Fungizone)
            --7504, --X nader te bepalen --non-stock medication
            8127, --Meropenem (Meronem)
            8229, --Myambutol (ethambutol)
            8374, --Kinine dihydrocloride
            --8375, --Immunoglobuline (Nanogam) -> not anbiotic
            --8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
            8547, --Voriconazol(VFEND)
            --9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
            9030, --Aztreonam (Azactam)
            9047, --Chlooramfenicol
            --9075, --Fusidinezuur (Fucidin) -> prophylaxis
            9128, --Piperacilline (Pipcil)
            9133, --Ceftriaxon (Rocephin)
            --9151, --Cefuroxim (Zinacef) -> often used for GI/transplant surgical prophylaxis
            --9152, --Cefazoline (Kefzol) -> prophylaxis for cardiac surgery
            9458, --Caspofungine
            9542, --Itraconazol (Trisporal)
            --9602, --Tetanusimmunoglobuline -> prophylaxis/not antibiotic
            12398, --Levofloxacine (Tavanic)
            12772, --Amfotericine B lipidencomplex  (Abelcet)
            15739, --Ecalta (Anidulafungine)
            16367, --Research Anidulafungin/placebo
            16368, --Research Caspofungin/placebo
            18675, --Amfotericine B in liposomen (Ambisome )
            19137, --Linezolid (Zyvoxid)
            19764, --Tigecycline (Tygacil)
            19773, --Daptomycine (Cubicin)
            20175 --Colistine
        )
        AND start < 6*60*60*1000 --within 6 hours (to correct for antibiotics administered before ICU)
    GROUP BY admissionid
), other_antibiotics AS ( --'prophylactic' antibiotics that may be used for sepsis
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS other_antibiotics_bool,
        STRING_AGG(DISTINCT item, '; ') AS other_antibiotics_given
    FROM drugitems
    WHERE 
        itemid IN (
            7064, --Vancomycine -> prophylaxis for valve surgery
            7187, --Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
            8394, --Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
            9029, --Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
            9151, --Cefuroxim (Zinacef) -> often used for GI surgical prophylaxis
            9152 --Cefazoline (Kefzol) -> prophylaxis
        )
        AND start < 24*60*60*1000 --within 24 hours (to correct for antibiotics administered before ICU)
    GROUP BY admissionid    
), cultures AS (
    SELECT
        admissionid,
        CASE 
            WHEN COUNT(*) > 0 THEN 1
            ELSE 0
        END AS sepsis_cultures_bool,
        STRING_AGG(DISTINCT item, '; ') AS sepsis_cultures_drawn
    FROM procedureorderitems
    WHERE 
        itemid IN (
        --8097, --Sputumkweek afnemen -> often used routinely
        --8418, --Urinekweek afnemen
        --8588, --MRSA kweken afnemen 
        9189, --Bloedkweken afnemen
        9190, --Cathetertipkweek afnemen
        --9191, --Drainvochtkweek afnemen
        --9192, --Faeceskweek afnemen -> Clostridium
        --9193, --X-Kweek nader te bepalen
        --9194, --Liquorkweek afnemen
        --9195, --Neuskweek afnemen
        --9197, --Perineumkweek afnemen -> often used routinely
        -9198, --Rectumkweek afnemen -> often used routinely
        9200, --Wondkweek afnemen
        9202, --Ascitesvochtkweek afnemen
        --9203, --Keelkweek afnemen -> often used routinely
        --9204, --SDD-kweken afnemen -> often used routinely
        9205 --Legionella sneltest (urine)
        --1302, --SDD Inventarisatiekweken afnemen -> often used routinely
        --19663, --Research Neuskweek COUrSe
        --19664, --Research Sputumkweek COUrSe
        )
        AND registeredat < 6*60*60*1000 --within 6 hours
    GROUP BY admissionid
)
SELECT 
    admissions.* 
    , diagnosis_type
    , diagnosis, diagnosis_id
    , diagnosis_subgroup
    , diagnosis_subgroup_id
    , diagnosis_group
    , diagnosis_group_id
    , surgical
    , sepsis_at_admission
    , sepsis_antibiotics_bool
    , sepsis_antibiotics_given
    , other_antibiotics_bool
    , other_antibiotics_given
    , sepsis_cultures_bool
    , sepsis_cultures_drawn
FROM admissions
LEFT JOIN diagnoses on admissions.admissionid = diagnoses.admissionid
LEFT JOIN diagnosis_subgroups on admissions.admissionid = diagnosis_subgroups.admissionid
LEFT JOIN diagnosis_groups on admissions.admissionid = diagnosis_groups.admissionid
LEFT JOIN sepsis on admissions.admissionid = sepsis.admissionid
LEFT JOIN sepsis_antibiotics on admissions.admissionid = sepsis_antibiotics.admissionid
LEFT JOIN other_antibiotics on admissions.admissionid = other_antibiotics.admissionid
LEFT JOIN cultures on admissions.admissionid = cultures.admissionid
WHERE --only last updated record
    (diagnoses.rownum = 1 OR diagnoses.rownum IS NULL) AND 
    (diagnosis_subgroups.rownum = 1 OR diagnosis_subgroups.rownum IS NULL) AND
    (diagnosis_groups.rownum = 1 OR diagnosis_groups.rownum IS NULL) AND
    (sepsis.rownum = 1 OR sepsis.rownum IS NULL) 
;
'''

**Load admissions table and dictionary**

In [8]:
admissions = pd.read_gbq(
  '''
  SELECT * FROM admissions;
  '''
  , configuration = config_gbq)
admissions.head()

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty
0,117,130,1,MC,0,None,0,2003-2009,91560000,26,None,Man,70-79,NaN,70-79,None,180-189,None,None
1,570,656,1,MC,0,None,0,2003-2009,90780000,25,None,Man,70-79,2.294424e+11,60-69,None,160-169,None,None
2,700,799,1,MC,0,None,0,2003-2009,104460000,29,None,Man,60-69,NaN,70-79,None,170-179,None,None
3,817,941,1,MC,0,None,0,2003-2009,86580000,24,None,Man,70-79,1.626498e+11,60-69,None,160-169,None,None
4,951,1098,1,MC,0,None,0,2003-2009,75000000,20,None,,70-79,NaN,60-69,None,170-179,None,None


In [9]:
# Total number of admissions
admissions['patientid'].nunique()

20109

In [10]:
#get the amsterdamumcdb package from PiPy repository for use in Colab
!pip install amsterdamumcdb
import amsterdamumcdb as adb
dictionary = adb.get_dictionary()

     |████████████████████████████████| 204kB 6.9MB/s 


**Load atrial fibrillation and define our subset of patients**

In [11]:
af = pd.read_gbq(
    '''
    SELECT
      admissionid
      ,itemid
      --,item
      ,valueid
      --,value
      ,measuredat
      --,registeredat
      --,registeredby
      --,updatedat
      --,updatedby
      --,islabresult
    FROM listitems
    WHERE 
      itemid = 6671	--Hartritme
      AND valueid = 13	--Atr fib
    '''
    , configuration=config_gbq)

In [12]:
# Total number of AF patients
admissions.loc[admissions['admissionid'].isin(af['admissionid']), 'patientid'].nunique()

3672

In [13]:
af_inclusion_tf = (af['measuredat'] > 0.5 * 60 * 60 * 1000) & (af['measuredat'] < 5 * 24 * 60 * 60 * 1000)
af_exclusion_tf = af['measuredat'] < 0

In [14]:
include_admissionid = af.loc[af_inclusion_tf, 'admissionid']
# Total number of AF patients with 0.5hr < AF < 24hr
admissions.loc[admissions['admissionid'].isin(include_admissionid), 'patientid'].nunique()

3073

In [15]:
exclude_admissionid = af.loc[af_exclusion_tf, 'admissionid']
include_admissionid = include_admissionid[~include_admissionid.isin(exclude_admissionid)]
# Total number of AF patients with no AF < 0hr
admissions.loc[admissions['admissionid'].isin(include_admissionid), 'patientid'].nunique()

2790

In [16]:
admissions.loc[admissions['admissionid'].isin(include_admissionid) & admissions['admissioncount'] == 1, 'admissionid'].nunique()

2683

**Admission diagnoses**

In [17]:
combined_diagnoses = pd.read_gbq(diagnoses_sql, configuration = config_gbq)
combined_diagnoses.head()

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty,diagnosis_type,diagnosis,diagnosis_id,diagnosis_subgroup,diagnosis_subgroup_id,diagnosis_group,diagnosis_group_id,surgical,sepsis_at_admission,sepsis_antibiotics_bool,sepsis_antibiotics_given,other_antibiotics_bool,other_antibiotics_given,sepsis_cultures_bool,sepsis_cultures_drawn
0,3724,4291,2,IC,1,Special/Medium care zelfde ziekenhuis,18900000,2010-2016,437940000,117,None,Man,70-79,NaN,90-99,None,170-179,None,Neurochirurgie,NICE APACHE IV,Non-operative neurologic - Seizures (primary-n...,143.0,Vasculair chirurgisch,1.0,Non-operative neurologic - Seizures (primary-n...,143.0,0.0,0.0,NaN,None,NaN,None,NaN,None
1,338,385,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,69840000,19,2,Man,18-39,NaN,None,None,None,None,Inwendig,NICE APACHE IV,"Non-operative neurologic - Overdose, other tox...",139.0,Overige,8.0,"Non-operative neurologic - Overdose, other tox...",139.0,0.0,0.0,NaN,None,NaN,None,NaN,None
2,398,461,3,IC,1,Verpleegafdeling zelfde ziekenhuis,51306240000,2010-2016,51337320000,8,2,Man,70-79,5.160114e+10,None,None,None,None,Heelkunde Gastro-enterologie,NICE APACHE IV,Post-operative gastro-intestinal - Complicatio...,296.0,Buikchirurgie,9.0,Post-operative gastro-intestinal - Complicatio...,296.0,1.0,NaN,NaN,None,NaN,None,NaN,None
3,451,517,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,103680000,29,2,Man,18-39,NaN,70-79,None,170-179,None,Traumatologie,NICE APACHE IV,Non-operative respiratory - Obstruction-airway...,161.0,Pulmonaal,2.0,Non-operative respiratory - Obstruction-airway...,161.0,0.0,0.0,1.0,Ceftriaxon (Rocephin),NaN,None,NaN,None
4,2419,2803,1,MC,1,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,228060000,63,2,Man,18-39,NaN,100-109,None,190+,None,Traumatologie,NICE APACHE IV,Non-operative trauma - Extremity only trauma,198.0,Traumatologie,5.0,Non-operative trauma - Extremity only trauma,198.0,0.0,0.0,NaN,None,NaN,None,NaN,None


In [36]:
combined_diagnoses['patientid'].nunique()

20109

**Identify subset of patients with AF**

In [18]:
af_admission = combined_diagnoses.loc[combined_diagnoses['admissionid'].isin(include_admissionid) & combined_diagnoses['admissioncount'] == 1, :]
print(af_admission.shape)
af_admission.head()

(2683, 34)


,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty,diagnosis_type,diagnosis,diagnosis_id,diagnosis_subgroup,diagnosis_subgroup_id,diagnosis_group,diagnosis_group_id,surgical,sepsis_at_admission,sepsis_antibiotics_bool,sepsis_antibiotics_given,other_antibiotics_bool,other_antibiotics_given,sepsis_cultures_bool,sepsis_cultures_drawn
9,4237,4869,1,MC,1,Eerste Hulp afdeling ander ziekenhuis,0,2010-2016,242520000,67,2,Man,80+,1.659060e+09,59-,None,180-189,None,"Maag-,Darm-,Leverziekten",NICE APACHE IV,"Non-operative Gastro-intestinal - Bleeding, up...",51.0,Abdominaal,3.0,"Non-operative Gastro-intestinal - Bleeding, up...",51.0,0.0,0.0,NaN,None,NaN,None,NaN,None
16,11146,12916,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,9840000,3,2,Man,70-79,1.866000e+07,None,None,None,None,None,NICE APACHE IV,Non-operative neurologic - Nontraumatic coma d...,135.0,Vasculair neurologisch,1.0,Non-operative neurologic - Nontraumatic coma d...,135.0,0.0,0.0,NaN,None,NaN,None,NaN,None
20,17323,20071,1,IC,1,Eerste Hulp afdeling ander ziekenhuis,0,2010-2016,35400000,10,2,Vrouw,70-79,NaN,60-69,None,160-169,None,Longziekte,NICE APACHE IV,Non-operative respiratory - Smoke inhalation,171.0,Traumatologie,5.0,Non-operative respiratory - Smoke inhalation,171.0,0.0,0.0,NaN,None,NaN,None,NaN,None
32,18430,21345,1,IC&MC,1,Eerste Hulp afdeling ander ziekenhuis,0,2010-2016,1248600000,347,3,Vrouw,60-69,NaN,60-69,None,160-169,None,Heelkunde Gastro-enterologie,NICE APACHE IV,Post-operative gastro-intestinal - Esophageal ...,298.0,Buikchirurgie,9.0,Post-operative gastro-intestinal - Esophageal ...,298.0,1.0,0.0,NaN,None,1.0,Metronidazol (Flagyl),NaN,None
100,6582,7639,1,IC,0,CCU/IC zelfde ziekenhuis,0,2010-2016,740640000,206,9,Man,70-79,1.129080e+09,None,None,None,None,Cardiologie,NICE APACHE IV,"Non-operative cardiovascular - Shock, cardiogenic",41.0,Cardiovasculair,1.0,"Non-operative cardiovascular - Shock, cardiogenic",41.0,0.0,0.0,NaN,None,NaN,None,NaN,None


**Identify cardio-thoracic patients**

In [19]:
af_admission['is_cardiothoracic'] = af_admission['specialty'].isin(['Cardiochirurgie', 'Cardiologie'])
# Number of patients from cardiology or cardiac surgery
af_admission['is_cardiothoracic'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


1153

In [20]:
# Number of patients with emergency (unplanned) cardiothoracic admission
(af_admission['is_cardiothoracic'] & af_admission['urgency']).sum()

307

In [21]:
re_cardiosurg = r'(CABG|AVR|hartchirurgie|heart surgery|Chron. cardiovasculaire ziekte|hartkleppen' + \
r'|cardiovascula|MVP|MVR|mitral|tricuspid|pericard|aortic.*valve|lobectom|segment|thorax|Bentall|aorta-ascendens|aorta-boog' + \
r'|aorta-wortel|aorta-descendens|lung|pneumectomie|bullectom|respiratoir neoplasm|thoracoscop|thoracotom(y|ie)|respirato' + \
r'|vrije wand ruptuur|VSR|ASD|pleurectom|intracardiac|aneurysmectom|congenital defect repair)(?! for esophag)'
re_cardiosurg = r'(CABG|AVR|MVP|MVR|valve)'

In [22]:
cardiosurg = af_admission[
    (af_admission['surgical'] == 1) 
    & (af_admission['diagnosis'].str.contains(re_cardiosurg, na=False, flags=re.IGNORECASE))]
cardiosurg.loc[:, 'diagnosis'].size

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


453

In [23]:
# cardiosurg['diagnosis'].value_counts()
af_admission['is_cardiosurg'] = af_admission['patientid'].isin(cardiosurg['patientid'])
# Patients who have CBAG / valve surgery (from cardiology or cardiac surgery)
(af_admission['is_cardiothoracic'] & af_admission['is_cardiosurg']).sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


452

In [24]:
# Number with emergency CABG/valve surgery + NOAF
(af_admission['is_cardiosurg'] & af_admission['urgency']).sum()

25

In [83]:
# Number with CABG/valve surgery, all patients
combined_diagnoses.loc[(combined_diagnoses['surgical'] == 1) & (combined_diagnoses['diagnosis'].str.contains(re_cardiosurg, na=False, flags=re.IGNORECASE)), 'patientid'].nunique()

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


4768

In [84]:
combined_diagnoses.loc[combined_diagnoses['specialty'].isin(['Cardiochirurgie', 'Cardiologie']), 'patientid'].nunique()

8367

**Identify patients with sepsis**

In [25]:
re_sepsis_surg = r'sepsis|pneumoni|GI perforation|perforation/rupture|infection|abscess|GI Vascular ischemia|diverticular|appendectomy|peritonitis'
re_sepsis_med = r'sepsis|septic|infect|pneumoni|cholangitis|pancr|endocarditis|meningitis|GI perforation|abces|abscess|darm ischaemie|GI vascular|fasciitis' + \
r'|inflammatory|peritonitis'

In [26]:
# Sepsis for surgical reasons
af_admission['sepsis_surgical'] = (af_admission['surgical'] == 1) & (af_admission['diagnosis'].str.contains(re_sepsis_surg, na=False, flags=re.IGNORECASE))
af_admission['sepsis_surgical'] &= ~(af_admission['sepsis_at_admission'] == 0)
af_admission['sepsis_surgical'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


22

In [27]:
# Sepsis for surgical reasons + endocarditis
(af_admission['sepsis_surgical'] & af_admission['diagnosis'].str.contains('endocarditis', na = False, flags = re.IGNORECASE)).sum()

0

In [28]:
af_admission['sepsis_admission'] = (af_admission['surgical'] == 0) & (af_admission['diagnosis'].str.contains(re_sepsis_med, na=False, flags=re.IGNORECASE))
af_admission['sepsis_admission'] &= ~(af_admission['sepsis_at_admission'] == 0)
af_admission['sepsis_admission'] |= (af_admission['sepsis_at_admission'] == 1)
af_admission['sepsis_admission'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

250

In [29]:
af_admission['sepsis_antibiotics'] = (af_admission['sepsis_antibiotics_bool'] == 1)
af_admission['sepsis_antibiotics'] &= ~(af_admission['sepsis_at_admission'] == 0)
af_admission['sepsis_antibiotics'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


480

In [30]:
af_admission['sepsis_culture'] = (af_admission['other_antibiotics_bool'] == 1) & (af_admission['sepsis_cultures_bool'] == 1)
af_admission['sepsis_culture'] &= ~(af_admission['sepsis_at_admission'] == 0)
af_admission['sepsis_culture'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


47

In [50]:
# All sepsis
(af_admission['sepsis_surgical'] | af_admission['sepsis_admission'] | af_admission['sepsis_antibiotics'] | af_admission['sepsis_culture']).sum()

559

In [32]:
# Sepsis antibiotics / culture
(af_admission['sepsis_antibiotics'] | af_admission['sepsis_culture']).sum()

495

In [33]:
((af_admission['sepsis_antibiotics'] | af_admission['sepsis_culture']) & af_admission['sepsis_admission']).sum()

186

In [44]:
# Total sepsis
combined_diagnoses.loc[
    (
        (
            #use reasons for admission
            #surgical admissions with sepsis
            (combined_diagnoses['surgical'] == 1) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_surg, na=False, flags=re.IGNORECASE))
        # ) | (
        #     #medical admissions with sepsis
        #     (combined_diagnoses['surgical'] == 0) 
        #     & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_med, na=False, flags=re.IGNORECASE))
        # ) | (
        #     #uses documentation at admission form (Early Goal Directed Therapy)
        #     (combined_diagnoses['sepsis_at_admission'] == 1)
        # ) | (
        #     #uses administered (therapeutic) antibiotics for determining sepsis
        #     (combined_diagnoses['sepsis_antibiotics_bool'] == 1)
        # ) | (
        #     #uses combination of administered antibiotics (that sometimes are used as prophylaxis) AND 
        #     #drawn cultures for determining sepsis
        #     (combined_diagnoses['other_antibiotics_bool'] == 1) 
        #     & (combined_diagnoses['sepsis_cultures_bool'] == 1)
        )
    ) & ~(combined_diagnoses['sepsis_at_admission'] == 0)
    #exclude all diagnoses where explicitly 'no sepsis' was documented
    , 'patientid'].nunique()

101

**Both sepsis and CABG / valve surgery**

In [49]:
((af_admission['sepsis_surgical'] | af_admission['sepsis_admission'] | af_admission['sepsis_antibiotics'] | af_admission['sepsis_culture']) & af_admission['is_cardiosurg']).sum()

4

# **Desciptive stats**

In [104]:
ds_rows = ['No. patients']
ds_rows += ['Age: ' + agegroup + ' %' for agegroup in af_admission['agegroup'].unique()]
ds_rows += ['Female %', 'Male %', 'Urgency %', 'BMI *', 'BMI **']
ds_rows += ['Weight >100kg %', 'Admission SOFA (to do)', 'From surgery %']
#ds_rows += ['Chronic condition %'] # may not be possible
ds_rows += ['ICU length of stay (days)']
#ds_rows += ['Pre-ICU length of stay (days)'] # may not be possible
ds_rows += ['Mortality %']

ds_columns = ['All', 'Sepsis subset', 'CABG/valve subset']
ds_columns += ['AF subset', 'AF+Sepsis', 'AF+CABG/valve']

table = pd.DataFrame(None, columns = ds_columns, index = ds_rows)


In [78]:
weight_mean_dict = {'59-': 59, '60-69': 65, '70-79': 75, '80-89': 85, '90-99': 95, '100-109': 105, '110+': 110}
weight_max_dict = {'59-': 59, '60-69': 69, '70-79': 79, '80-89': 89, '90-99': 99, '100-109': 109, '110+': 110}
height_mean_dict = {'159-': 1.59, '160-169': 1.65, '170-179': 1.75, '180-189': 1.85, '190+': 1.90}
height_min_dict = {'159-': 1.59, '160-169': 1.60, '170-179': 1.70, '180-189': 1.80, '190+': 1.90}

combined_diagnoses['weightmean'] = combined_diagnoses['weightgroup'].copy().replace(weight_mean_dict)
combined_diagnoses['weightmax'] = combined_diagnoses['weightgroup'].copy().replace(weight_max_dict)
combined_diagnoses['heightmean'] = combined_diagnoses['heightgroup'].copy().replace(height_mean_dict)
combined_diagnoses['heightmin'] = combined_diagnoses['heightgroup'].copy().replace(height_min_dict)

af_admission['weightmean'] = af_admission['weightgroup'].copy().replace(weight_mean_dict)
af_admission['weightmax'] = af_admission['weightgroup'].copy().replace(weight_max_dict)
af_admission['heightmean'] = af_admission['heightgroup'].copy().replace(height_mean_dict)
af_admission['heightmin'] = af_admission['heightgroup'].copy().replace(height_min_dict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [86]:
all_ind = combined_diagnoses['patientid'].isin(combined_diagnoses['patientid'])
sepsis_ind =     (
        (
            #use reasons for admission
            #surgical admissions with sepsis
            (combined_diagnoses['surgical'] == 1) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_surg, na=False, flags=re.IGNORECASE))
        ) | (
            #medical admissions with sepsis
            (combined_diagnoses['surgical'] == 0) 
            & (combined_diagnoses['diagnosis'].str.contains(re_sepsis_med, na=False, flags=re.IGNORECASE))
        ) | (
            #uses documentation at admission form (Early Goal Directed Therapy)
            (combined_diagnoses['sepsis_at_admission'] == 1)
        ) | (
            #uses administered (therapeutic) antibiotics for determining sepsis
            (combined_diagnoses['sepsis_antibiotics_bool'] == 1)
        ) | (
            #uses combination of administered antibiotics (that sometimes are used as prophylaxis) AND 
            #drawn cultures for determining sepsis
            (combined_diagnoses['other_antibiotics_bool'] == 1) 
            & (combined_diagnoses['sepsis_cultures_bool'] == 1)
        )
    ) & ~(combined_diagnoses['sepsis_at_admission'] == 0)
cardiosurg_ind = (combined_diagnoses['surgical'] == 1
                  ) & (combined_diagnoses['diagnosis'].str.contains(re_cardiosurg, na=False, flags=re.IGNORECASE))

af_ind = af_admission['patientid'].isin(af_admission['patientid'])
af_sepsis_ind = (af_admission['sepsis_surgical'] | 
                 af_admission['sepsis_admission'] | 
                 af_admission['sepsis_antibiotics'] | 
                 af_admission['sepsis_culture'])
af_cardiosurg_ind = (af_admission['is_cardiothoracic'] & af_admission['is_cardiosurg'])

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [105]:
conditions = [all_ind, sepsis_ind, cardiosurg_ind, af_ind, af_sepsis_ind, af_cardiosurg_ind]
for ii in range(6):
  col = ds_columns[ii]
  if ii < 3:
    subset = combined_diagnoses.loc[conditions[ii], :]
  else:
    subset = af_admission.loc[conditions[ii], :]
  table.loc['No. patients', col] = subset['patientid'].nunique()
  for agegroup in af_admission['agegroup'].unique():
    table.loc['Age: ' + agegroup + ' %', col] = (subset['agegroup'] == agegroup).mean() * 100
  table.loc['Female %', col] = (subset['gender'] == 'Vrouw').mean() * 100
  table.loc['Male %', col] = (subset['gender'] == 'Man').mean() * 100
  table.loc['Urgency %', col] = (subset['urgency'] == 1).mean() * 100
  table.loc['BMI *', col] = (subset['weightmean'] / (subset['heightmean'] ** 2)).mean()
  table.loc['BMI **', col] = (subset['weightmax'] / (subset['heightmin'] ** 2)).mean()
  table.loc['Weight >100kg %', col] = subset['weightgroup'].isin(['100-109', '110+']).mean() * 100
  table.loc['From surgery %', col] = (subset['surgical'] == 1).mean()
  table.loc['ICU length of stay (days)', col] = subset['lengthofstay'].mean() / 24
  table.loc['Mortality %', col] = (~subset['dateofdeath'].isna()).mean() * 100
table

,All,Sepsis subset,CABG/valve subset,AF subset,AF+Sepsis,AF+CABG/valve
No. patients,20109,2401,4768,2683,559,452
Age: 80+ %,9.78534,13.4669,9.42768,23.8539,26.1181,19.469
Age: 70-79 %,26.2875,22.8858,34.7763,39.7316,34.347,50.2212
Age: 60-69 %,26.3611,25.491,34.6098,23.3321,23.4347,23.0088
Age: 50-59 %,17.1038,16.6733,15.9001,9.05703,10.7335,6.19469
Age: 40-49 %,9.4218,10.3006,4.39126,2.27357,3.04114,0.663717
Age: 18-39 %,11.0404,11.1824,0.894901,1.75177,2.32558,0.442478
Female %,34.0821,39.0381,25.5359,35.8181,35.4204,34.5133
Male %,63.7713,58.2766,74.4433,61.6101,61.0018,65.4867
Urgency %,27.0319,36.1122,3.43392,34.5509,36.4937,5.53097


**APACHE II / SOFA**

Past medical history?

In [34]:
import re
# dictionary.loc[dictionary['abbreviation'].str.contains(r'BMI', flags = re.IGNORECASE),:]
dictionary.head()
# print('\n'.join(dictionary.loc[dictionary['category'].str.contains(r'apache ii', flags = re.IGNORECASE) & (dictionary['count'] > 0),'value'].unique().astype(str)))
dictionary.loc[dictionary['item'].str.contains(r'Apache II Non-Operatief Cardiovasculair', flags = re.IGNORECASE),:].head(5)
# dictionary.loc[dictionary['item'].str.contains(r'Ritmestoornissen', flags = re.IGNORECASE), :]

,itemid,item,item_en,abbreviation,categoryid,category,category_en,ordercategoryid,ordercategory,islabresult,valueid,value,unitid,unit,low_normal_value,high_normal_value,table,count,count_validated
243,18589,Apache II Non-Operatief Cardiovasculair,NaN,Apache II Non-Operatief Cardiovasculair,707,APACHE II Diagnosen,NaN,NaN,NaN,0.0,1.0,Cardiovasculair,NaN,NaN,NaN,NaN,listitems,276,276.0
244,18589,Apache II Non-Operatief Cardiovasculair,NaN,Apache II Non-Operatief Cardiovasculair,707,APACHE II Diagnosen,NaN,NaN,NaN,0.0,2.0,Cardiovasculair multiple trauma,NaN,NaN,NaN,NaN,listitems,82,82.0
245,18589,Apache II Non-Operatief Cardiovasculair,NaN,Apache II Non-Operatief Cardiovasculair,707,APACHE II Diagnosen,NaN,NaN,NaN,0.0,3.0,Coronair lijden,NaN,NaN,NaN,NaN,listitems,104,104.0
246,18589,Apache II Non-Operatief Cardiovasculair,NaN,Apache II Non-Operatief Cardiovasculair,707,APACHE II Diagnosen,NaN,NaN,NaN,0.0,4.0,Thoracaal of abdominaal aneurysma,NaN,NaN,NaN,NaN,listitems,62,62.0
247,18589,Apache II Non-Operatief Cardiovasculair,NaN,Apache II Non-Operatief Cardiovasculair,707,APACHE II Diagnosen,NaN,NaN,NaN,0.0,5.0,Congestief hart falen,NaN,NaN,NaN,NaN,listitems,294,294.0


In [35]:
# apache = pd.read_gbq(
#     '''
#     SELECT
#       admissionid
#       ,itemid
#       --,item
#       ,valueid
#       ,value
#       ,measuredat
#       --,registeredat
#       --,registeredby
#       --,updatedat
#       --,updatedby
#       --,islabresult
#     FROM listitems
#     WHERE 
#       itemid = 18589	--Apache II Non-Operatief Cardiovasculair
#     '''
#     , configuration=config_gbq)